# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The business objective of this task is to optimize the effectiveness of a Portuguese banking institution's direct marketing campaigns. Specifically, the goal is to:

Predict Customer Subscription: Build a predictive model to determine whether a client will subscribe to a term deposit (indicated by the target variable $y$).

Optimize Marketing Resources: By identifying clients with a high probability of subscribing, the bank can focus its resources (time, staffing, and marketing budget) on the most promising leads, thereby reducing the number of unnecessary or unproductive contacts.

Identify Key Success Factors: Analyze client demographics (age, job, education), past campaign outcomes, and socio-economic indicators (employment variation rate, consumer price index) to understand which factors most significantly influence a client's decision to subscribe.

Ultimately, this allows the bank to increase its conversion rate for term deposits while maintaining cost-efficient marketing operations.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv("https://raw.githubusercontent.com/maniliy07/Module17/refs/heads/main/bank-additional-full.csv",sep=';')

# Inspect the first few rows and summary information
print(df.head())
print(df.info())
print(df['y'].value_counts())

   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx  euribor3m  nr.employed

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
# Check for explicit missing values
null_counts = df.isnull().sum()

# Check for 'unknown' strings which often represent missing data in this specific dataset
unknown_counts = (df == 'unknown').sum()

# Display data types and unique values for categorical columns to check for coercion needs
data_types = df.dtypes
categorical_summary = {col: df[col].unique()[:5] for col in df.select_dtypes(include=['object']).columns}

print("Explicit Null Counts:\n", null_counts)
print("\n'Unknown' Value Counts:\n", unknown_counts)
print("\nData Types:\n", data_types)

Explicit Null Counts:
 age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

'Unknown' Value Counts:
 age                  0
job                330
marital             80
education         1731
default           8597
housing            990
loan               990
contact              0
month                0
day_of_week          0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
emp.var.rate         0
cons.price.idx       0
cons.conf.idx        0
euribor3m            0
nr.employed          0
y                    0
dtype: int64

Data Types:
 age     

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

The Business Objective of this task is to develop a predictive model that identifies the likelihood of a client subscribing to a term deposit following a direct marketing (telemarketing) campaign.

By accurately predicting the outcome ($y$), the bank can achieve the following goals

:Increase Campaign Efficiency: Focus marketing efforts and resources on clients with the highest probability of conversion, rather than contacting the entire database.

Reduce Operational Costs: Minimize the time and labor costs spent on "unsuccessful" calls.

Enhance Customer Experience: Reduce "marketing fatigue" by avoiding unnecessary contact with customers who have a low propensity to subscribe.

Identify Key Drivers: Determine which factors (e.g., economic indicators like the euribor3m rate, or customer demographics like job and education) are the strongest predictors of a successful subscription to inform future product offerings and strategy.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [5]:


# Select the bank information features (1-7) and the target (y)
bank_features = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
target = 'y'

X = df[bank_features].copy()
y = df[target].copy()

# 1. Encode the target variable (yes -> 1, no -> 0)
y = y.map({'yes': 1, 'no': 0})

# 2. Encode categorical features
# Using get_dummies for job, marital, education, default, housing, loan
# This handles the 'unknown' values as distinct categories.
X_encoded = pd.get_dummies(X, columns=['job', 'marital', 'education', 'default', 'housing', 'loan'], drop_first=True)

# Combine into a single dataframe for the final output
processed_df = pd.concat([X_encoded, y], axis=1)

# Save the processed data to CSV
processed_df.to_csv('processed_bank_data.csv', index=False)

# Display the first few rows and column names to confirm
print("Encoded columns:", processed_df.columns.tolist())
print(processed_df.head())

Encoded columns: ['age', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'marital_married', 'marital_single', 'marital_unknown', 'education_basic.6y', 'education_basic.9y', 'education_high.school', 'education_illiterate', 'education_professional.course', 'education_university.degree', 'education_unknown', 'default_unknown', 'default_yes', 'housing_unknown', 'housing_yes', 'loan_unknown', 'loan_yes', 'y']
   age  job_blue-collar  job_entrepreneur  job_housemaid  job_management  \
0   56            False             False           True           False   
1   57            False             False          False           False   
2   37            False             False          False           False   
3   40            False             False          False           False   
4   56            False             False          False           F

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [6]:
from sklearn.model_selection import train_test_split

#  Split features and target
X = processed_df.drop(columns=['y'])
y = processed_df['y']

# Split into train and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Save the sets to CSV
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

# Print shapes to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train distribution:\n{y_train.value_counts(normalize=True)}")
print(f"y_test distribution:\n{y_test.value_counts(normalize=True)}")

X_train shape: (28831, 28)
X_test shape: (12357, 28)
y_train distribution:
y
0    0.887343
1    0.112657
Name: proportion, dtype: float64
y_test distribution:
y
0    0.887351
1    0.112649
Name: proportion, dtype: float64


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

To establish a baseline for your classifier, we look at the performance of a Majority Class Classifier (also known as a "Zero Rule" model). This model always predicts the most frequent outcome regardless of the input data.


Based on the distribution of your training data:

"No" (Did not subscribe): ~88.73%

"Yes" (Subscribed): ~11.27%

Baseline Accuracy: 88.73%
If your model simply predicts "no" for every single client, it will be correct approximately 88.73% of the time. This is the score to beat.

Why this matters:
The Accuracy Trap: Any model that achieves an accuracy lower than or equal to 88.73% is effectively performing no better (or worse) than a model that does not learn anything from the features.

Beyond Accuracy: Because the dataset is imbalanced (only ~11% subscribed), accuracy alone is not a sufficient metric. While the baseline accuracy is high, the baseline Recall for the "yes" class is 0%. A useful business model must aim to beat the 88.7% accuracy while specifically maximizing the identification of the "yes" subscribers through metrics like F1-Score, Precision, and Recall.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the split data
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()

# Initialize and train the Logistic Regression model
# Increased max_iter to ensure convergence
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)


### Problem 9: Score the Model

What is the accuracy of your model?

In [8]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.8874

Classification Report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94     10965
           1       0.00      0.00      0.00      1392

    accuracy                           0.89     12357
   macro avg       0.44      0.50      0.47     12357
weighted avg       0.79      0.89      0.83     12357


Confusion Matrix:
 [[10965     0]
 [ 1392     0]]


c:\Users\test\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\test\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\test\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [9]:

import time
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:


# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()
}

results = []

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    train_time = time.time() - start_time
    
    train_accuracy = accuracy_score(y_train, model.predict(X_train_scaled))
    test_accuracy = accuracy_score(y_test, model.predict(X_test_scaled))
    
    results.append({
        'Model': name,
        'Train Time': train_time,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy
    })

# Create results DataFrame
results_df = pd.DataFrame(results)
print(results_df)

# Save to CSV for the user
results_df.to_csv('model_performance_comparison.csv', index=False)

                 Model  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.054186        0.887343       0.887351
1                  KNN    0.010412        0.890569       0.878530
2        Decision Tree    0.117302        0.918837       0.864045
3                  SVM   42.061559        0.887448       0.887432


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric


To move beyond default settings and improve model performance, we can implement Hyperparameter Tuning using GridSearchCV. This process systematically searches through a range of values to find the combination that maximizes your chosen performance metric.

Since this is a classification task (predicting whether a customer will subscribe), accuracy might be misleading if the classes are imbalanced. We should consider metrics like Precision, Recall, or F1-score.

Here is how we can structure the improvement phase:

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Example for Decision Tree
dt_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('dtree', DecisionTreeClassifier())
])

param_grid = {
    'dtree__max_depth': [3, 5, 10, None],
    'dtree__min_samples_split': [2, 5, 10],
    'dtree__criterion': ['gini', 'entropy']
}

grid = GridSearchCV(dt_pipe, param_grid, cv=5, scoring='f1') # Tuning for F1-score

##### Questions